In [ ]:
!pip install torchsummary

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os

import torch
import cv2
import torchvision
from torchvision import transforms, datasets, models
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import math
from PIL import Image
import albumentations as albu
import albumentations.pytorch as pyalbu
from torchsummary import summary

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

print("Albumentations Version: {}".format(albu.__version__))
print("Torch Version: {}".format(torch.__version__))

class_dict = {0: 'Fish', 1: 'Flower', 2: 'Gravel', 3: 'Sugar'}

raw_image_shape = (1400, 2100)

In [ ]:
path = '../input/understanding_cloud_organization'
os.listdir(path)

train = pd.read_csv(f'{path}/train.csv')
sub = pd.read_csv(f'{path}/sample_submission.csv')

n_train = len(os.listdir(f'{path}/train_images'))
n_test = len(os.listdir(f'{path}/test_images'))
print(f'There are {n_train} images in train dataset')
print(f'There are {n_test} images in test dataset')

train['label'] = train['Image_Label'].apply(lambda x: x.split('_')[1])
train['im_id'] = train['Image_Label'].apply(lambda x: x.split('_')[0])


sub['label'] = sub['Image_Label'].apply(lambda x: x.split('_')[1])
sub['im_id'] = sub['Image_Label'].apply(lambda x: x.split('_')[0])

In [ ]:
train.head()

In [ ]:
print("Total Labels for each class")
print(train.loc[train.label.notnull(),'label'].value_counts())

print("\nOccurence of number of labels for each image")
print(train.groupby("im_id").count()['EncodedPixels'].value_counts())

In [ ]:
def rle_decode(mask_rle: str = '', shape: tuple = (1400, 2100)):
    '''
    Decode rle encoded mask.
    
    :param mask_rle: run-length as string formatted (start length)
    :param shape: (height, width) of array to return 
    Returns numpy array, 1 - mask, 0 - background
    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0] * shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape, order='F')

def make_mask(df: pd.DataFrame, image_name: str='img.jpg', shape: tuple = (1400, 2100)):
    """
    Create mask based on df, image name and shape.
    """
    encoded_masks = df.loc[df['im_id'] == image_name, 'EncodedPixels']
    masks = np.zeros((shape[0], shape[1], 4), dtype=np.float32)

    for idx, label in enumerate(encoded_masks.values):
        if label is not np.nan:
            mask = rle_decode(label)
            masks[:, :, idx] = mask
            
    return masks

In [ ]:
fig = plt.figure(figsize=(25, 21))
for j, im_id in enumerate(np.random.choice(train['im_id'].unique(), 4)):
    for i, (idx, row) in enumerate(train.loc[train['im_id'] == im_id].iterrows()):
        ax = fig.add_subplot(5, 4, j * 4 + i + 1, xticks=[], yticks=[])
        im = Image.open(f"{path}/train_images/{row['Image_Label'].split('_')[0]}")
        plt.imshow(im)
        mask_rle = row['EncodedPixels']
        try: # label might not be there!
            mask = rle_decode(mask_rle)
        except Exception as e:
            mask = np.zeros(raw_image_shape)
#             print(e)
        plt.imshow(mask, alpha=0.5, cmap='gray')
        ax.set_title(f"Image: {row['Image_Label'].split('_')[0]}. Label: {row['label']}")


In [ ]:
resize_shape = (320,640)
# resize_shape = tuple([(x / 10) for x in raw_image_shape])

print(resize_shape)

# Transformations for the train
train_trans = albu.Compose([
    albu.HorizontalFlip(),
    albu.Resize(*resize_shape),
    pyalbu.ToTensor()])

# Transformations for the validation & test sets
val_trans = albu.Compose([
#     albu.HorizontalFlip(),
    albu.Resize(*resize_shape),
    pyalbu.ToTensor()])

In [ ]:
class CloudDataset(torch.utils.data.Dataset):
    def __init__(self, df, transforms, datatype: str = 'train', img_ids: np.array = None):
        self.df = df
        if datatype != 'test':
            self.data_folder = f"{path}/train_images"
        else:
            self.data_folder = f"{path}/test_images"
        self.img_ids = img_ids
        self.transforms = transforms

    def __getitem__(self, idx):
        image_name = self.img_ids[idx]
        mask = make_mask(self.df, image_name)
        image_path = os.path.join(self.data_folder, image_name)
        img = cv2.imread(image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        augmented = self.transforms(image=img, mask=mask)
        img = augmented['image']
        mask = augmented['mask']
        mask = mask[0].permute(2, 0, 1)
        return img, mask

    def __len__(self):
        return len(self.img_ids)

In [ ]:
# Train Test Split
train_img_split = train.loc[train.EncodedPixels.notnull(),'im_id'].value_counts().reset_index()
train_images, val_images = train_test_split(train_img_split['index'],
                                            stratify = train_img_split['im_id'],
                                            test_size=0.2)

train_data = CloudDataset(df = train,
                          transforms = train_trans,
                          datatype = 'train',
                          img_ids = train_images.values
                         )

val_data = CloudDataset(df = train,
                          transforms = val_trans,
                          datatype = 'validation',
                          img_ids = val_images.values
                         )

image_datasets = {
    'train': train_data, 'val': val_data
}

batch_size = 8
num_workers = 0

dataloaders = {
    'train': torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=num_workers),
    'val': torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=True, num_workers=num_workers)
}

In [ ]:
batch = next(iter(train_data))
inputshape = batch[0].size()
print("Input Shape: {}\nOutput Shape: {}".format(inputshape,batch[1].size()))

In [ ]:
fig = plt.figure(figsize = [25,18])
num_images = 6
for nth in range(1, num_images + 1):
    batch = train_data[np.random.randint(len(train_data), size=1)[0]]
    batch_image = batch[0].numpy().transpose((1,2,0))
    batch_mask = batch[1].numpy()

    ax = plt.subplot(num_images//2, 2, nth)
    ax.axis('off')
    ax.imshow(batch_image, cmap='gray', interpolation='none')
    for i,color in enumerate(['prism', 'viridis', 'twilight', 'plasma']):
        mask = batch_mask[i]
        if (mask > 0).any():
            masked = np.ma.masked_where(mask == 0, mask)
            ax.imshow(masked, cmap=color, alpha=0.3)
            
plt.subplots_adjust(wspace=0.1, hspace=0.1)

In [ ]:
class Net(nn.Module):
    def __init__(self, n_channels, n_classes, dropout):
        super(Net, self).__init__()

        self.convin = nn.Conv2d(in_channels=n_channels, out_channels=64, kernel_size=5, stride=1, padding=2)
        self.convout = nn.Conv2d(64, n_classes, 1)
        
        
    def forward(self, x):
        x = self.convin(x)
        x = F.relu(x)
        x = self.convout(x)
        return x

net = Net(n_channels=3, n_classes=4, dropout = .4).to(device)
net

In [ ]:
import torch.nn.functional as F
import torch

def dice_loss(pred, target, smooth = 1.):
    pred = pred.contiguous()
    target = target.contiguous()    

    intersection = (pred * target).sum(dim=2).sum(dim=2)
    
    loss = (1 - ((2. * intersection + smooth) / (pred.sum(dim=2).sum(dim=2) + target.sum(dim=2).sum(dim=2) + smooth)))
    
    return loss.mean()

def calc_loss(pred, target, metrics, bce_weight=0.5):
    bce = F.binary_cross_entropy_with_logits(pred, target)
        
    pred = torch.sigmoid(pred)
    dice = dice_loss(pred, target)
    
    loss = bce * bce_weight + dice * (1 - bce_weight)
    
    metrics['bce'] += bce.data.cpu().numpy() * target.size(0)
    metrics['dice'] += dice.data.cpu().numpy() * target.size(0)
    metrics['loss'] += loss.data.cpu().numpy() * target.size(0)
    
    return loss

def print_metrics(metrics, epoch_samples, phase):    
    outputs = []
    for k in metrics.keys():
        outputs.append("{}: {:4f}".format(k, metrics[k] / epoch_samples))
        
    print("{}: {}".format(phase, ", ".join(outputs)))    

def train_model(model, optimizer, scheduler, num_epochs=25):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 1e10

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        
        since = time.time()

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                for param_group in optimizer.param_groups:
                    print("LR", param_group['lr'])
                    
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            metrics = defaultdict(float)
            epoch_samples = 0
            
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)    

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = calc_loss(outputs, labels, metrics)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        scheduler.step()
                # statistics
                epoch_samples += inputs.size(0)

            print_metrics(metrics, epoch_samples, phase)
            epoch_loss = metrics['loss'] / epoch_samples

            # deep copy the model
            if phase == 'val' and epoch_loss < best_loss:
                print("saving best model")
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())

        time_elapsed = time.time() - since
        print('{:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
            
    print('Best val loss: {:4f}'.format(best_loss))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
# torch.cuda.empty_cache()

# base_model = models.resnet18(pretrained=False)
# list(base_model.children())

# net = ResNetUNet(n_class = 4)
# net = UNet(n_channels=3, n_classes=4).to(device)
summary(net, input_size=tuple(inputshape))

In [ ]:
import torch
import torch.optim as optim
from torch.optim import lr_scheduler
from collections import defaultdict
import time
import copy

# freeze backbone layers
#for l in model.base_layers:
#    for param in l.parameters():
#        param.requires_grad = False

optimizer = optim.Adam(net.parameters(), lr=0.1)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

model = train_model(net, optimizer, exp_lr_scheduler, num_epochs=3)